# Ontology overview
This notebook contains code for converting go.obo file into dataframe object and export it as csv. There is also a function `check_all_paths`, which accepts as arguments previously loaded with obonet graph and list of gene ontology (GO) id's. Function looks for relations between all nodes in list and returns all of it in that format: [[(node1, node2, relation), node1, node2, relation], [...]].

For next improvments, there is need to:
- Convert code into object paradigm, and use ex. fastAPI for comunication with frontend
- Optimize `check_all_paths` function and use multithreading

In [1]:
import obonet
import networkx as nx
import pandas as pd

graph = obonet.read_obo('go.obo')

nodes_df = pd.DataFrame.from_dict(dict(graph.nodes(data=True)), orient='index').reset_index()
nodes_df = nodes_df.rename(columns={'index': 'id'})
print("Nodes DataFrame Head:")
print(nodes_df.head())

edges_df = nx.to_pandas_edgelist(graph)
print("Edges DataFrame Head:")
print(edges_df.head())


print("Final Nodes DataFrame:")
print(nodes_df.head())
print("Final Edges DataFrame:")
print(edges_df.head())

Nodes DataFrame Head:
           id                                               name  \
0  GO:0000001                          mitochondrion inheritance   
1  GO:0000002                   mitochondrial genome maintenance   
2  GO:0000006  high-affinity zinc transmembrane transporter a...   
3  GO:0000007  low-affinity zinc ion transmembrane transporte...   
4  GO:0000009             alpha-1,6-mannosyltransferase activity   

            namespace                                                def  \
0  biological_process  "The distribution of mitochondria, including t...   
1  biological_process  "The maintenance of the structure and integrit...   
2  molecular_function  "Enables the transfer of zinc ions (Zn2+) from...   
3  molecular_function  "Enables the transfer of a solute or solutes f...   
4  molecular_function  "Catalysis of the transfer of a mannose residu...   

                                             synonym  \
0             ["mitochondrial inheritance" EXACT []]   


In [2]:
nodes_df.head()

,id,name,namespace,def,synonym,is_a,xref,property_value,alt_id,created_by,creation_date,subset,relationship,intersection_of,comment,disjoint_from
0,GO:0000001,mitochondrion inheritance,biological_process,"""The distribution of mitochondria, including t...","[""mitochondrial inheritance"" EXACT []]","[GO:0048308, GO:0048311]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GO:0000002,mitochondrial genome maintenance,biological_process,"""The maintenance of the structure and integrit...",NaN,[GO:0007005],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GO:0000006,high-affinity zinc transmembrane transporter a...,molecular_function,"""Enables the transfer of zinc ions (Zn2+) from...","[""high affinity zinc uptake transmembrane tran...",[GO:0005385],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GO:0000007,low-affinity zinc ion transmembrane transporte...,molecular_function,"""Enables the transfer of a solute or solutes f...",NaN,[GO:0005385],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GO:0000009,"alpha-1,6-mannosyltransferase activity",molecular_function,"""Catalysis of the transfer of a mannose residu...","[""1,6-alpha-mannosyltransferase activity"" EXAC...",[GO:0000030],"[Reactome:R-HSA-449718 ""Addition of a third ma...",[term_tracker_item https://github.com/geneonto...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
nodes_df.to_csv('go_nodes.csv', index=False)

In [4]:
def check_all_shortest_paths(graph, ontology_ids):
    found_paths_with_relations = []
    for i, node1 in enumerate(ontology_ids):
        for node2 in ontology_ids[i+1:]:
            if nx.has_path(graph, node1, node2):
                path = nx.shortest_path(graph, node1, node2)
                path_with_relations = []
                
                for j in range(len(path) - 1):
                    start_node = path[j]
                    end_node = path[j + 1]
                    relations = graph[start_node][end_node]
                    relation_type = next(iter(relations), None)
                    path_with_relations.append((start_node, end_node, relation_type))
                
                found_paths_with_relations.append(path_with_relations)
             
    return found_paths_with_relations

In [17]:
ontology_ids = nodes_df['id'].sample(100).tolist()
found_paths_with_relations = check_all_shortest_paths(graph, ontology_ids)

import json
with open('found_paths_with_relations.json', 'w') as f:
    json.dump(found_paths_with_relations, f, indent=4)
found_paths_with_relations

[[('GO:0050610', 'GO:0030614', 'is_a'), ('GO:0030614', 'GO:0030613', 'is_a')],
 [('GO:1904890', 'GO:1904861', 'negatively_regulates')]]

In [26]:
import pandas as pd

def get_connections(df, start_node):
    connections = []
    visited = set()
    stack = [start_node]

    while stack:
        current_node = stack.pop()
        if current_node in visited:
            continue
        visited.add(current_node)
        
        row = df.loc[df['id'] == current_node]
        if row.empty:
            continue
        
        is_a_values = row['is_a'].values[0]
        if type(is_a_values) != float:
            for target_node in is_a_values:
                connections.append((current_node, target_node, 'is_a'))
                stack.append(target_node)
        
        relationship_values = row['relationship'].values[0]
        if type(relationship_values) != float:
            for relationship in relationship_values:
                rel_type, target_node = relationship.split(' ')
                connections.append((current_node, target_node, rel_type))
                stack.append(target_node)
    
    return connections

connections = []
for i in range(len(found_paths_with_relations)):
    a = get_connections(nodes_df, found_paths_with_relations[i][0][0])
    connections.extend(a)
print(connections)
print(len(connections))
connections = set(connections) # remove duplicates if there is any
print(len(connections))


[('GO:0050610', 'GO:0030614', 'is_a'), ('GO:0030614', 'GO:0030613', 'is_a'), ('GO:0030613', 'GO:0016491', 'is_a'), ('GO:0016491', 'GO:0003824', 'is_a'), ('GO:0003824', 'GO:0003674', 'is_a'), ('GO:1904890', 'GO:0051964', 'is_a'), ('GO:1904890', 'GO:1904889', 'is_a'), ('GO:1904890', 'GO:1904861', 'negatively_regulates'), ('GO:1904861', 'GO:0007416', 'is_a'), ('GO:0007416', 'GO:0034329', 'is_a'), ('GO:0007416', 'GO:0050808', 'is_a'), ('GO:0007416', 'GO:0007399', 'part_of'), ('GO:0007399', 'GO:0048731', 'is_a'), ('GO:0048731', 'GO:0048856', 'is_a'), ('GO:0048731', 'GO:0007275', 'part_of'), ('GO:0007275', 'GO:0032501', 'is_a'), ('GO:0007275', 'GO:0048856', 'is_a'), ('GO:0048856', 'GO:0032502', 'is_a'), ('GO:0032502', 'GO:0008150', 'is_a'), ('GO:0032501', 'GO:0008150', 'is_a'), ('GO:0050808', 'GO:0034330', 'is_a'), ('GO:0034330', 'GO:0016043', 'is_a'), ('GO:0016043', 'GO:0071840', 'is_a'), ('GO:0071840', 'GO:0009987', 'is_a'), ('GO:0009987', 'GO:0008150', 'is_a'), ('GO:0034329', 'GO:0022607'

In [27]:
import json
with open('connections.json', 'w') as f:
    json.dump(list(connections), f, indent=4)
